In [2]:
# 모듈 import
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from collections import Counter
from pykospacing import Spacing #pip install git+https://github.com/haven-jeon/PyKoSpacing.git 
import time

In [67]:
# 빈도분석 + 키워드 추출 함수

def load_area():
    df = pd.read_csv('해커톤경산시.csv', encoding='ms949')
    df_body = df.iloc[:, 1:2]
    body = df_body['사업명']
    return body

def load_stopwords():
    with open('stopwords.txt', 'r') as f:
        list_file = f.readlines()
    return list_file[0].split(",")

# 정규 표현식을 통해 한글 단어만 남기기 (이모티콘, 초성, 영어 제거)
def extract_word(text):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', text)
    return result

def make_wordlist(body,stopwords):
    result = []
    for i in body:
        #정규표현식 적용
        #print("데이터 정제 중....")
        words = pd.Series(i)
        words = words.apply(lambda x:extract_word(x))
        spacing = Spacing()
        words = words.apply(lambda x:spacing(x))
        
        #형태소 추출
        #print("형태소 추출 중....")
        okt = Okt()
        words = " ".join(words.tolist())
        words = okt.morphs(words,stem=True)
        
        #한글자 제거
        #print("한글자 제거 중....")
        words = [x for x in words if len(x)>1]
        
        #불용어 제거
        #print("불용어 제거 중....")
        words = [x for x in words if x not in stopwords]
        
        # 형태소 분석 결과를 리스트에 저장
        result.append(words)
        
    return result


def run():
    body = load_area()
    print(make_wordlist(body, load_stopwords()))

In [68]:
run()

[['전기자동차', '보급', '사업', '하반기', '시행', '공고'], ['미디어', '콘텐츠', '산업', '육성', '사업'], ['경북', '청년', '키친', '조성', '운영'], ['주산지', '일관', '기계화', '농기', '지원', '사업', '신청', '모집', '공고'], ['경로당', '운영', '지원'], ['노인', '일자리', '사회', '활동', '지원', '사업'], ['장애인', '휠체어', '수리', '비용', '지원'], ['청년', '취업', '날개', '디네'], ['임산부', '건강', '관리', '지원'], ['신혼부부', '임차보증금', '이자', '지원'], ['영유', '보육료', '지원'], ['영아', '수당', '지원'], ['귀농', '정착', '지원', '사업'], ['다자', '지원', '사업'], ['경산시', '긴급', '재난', '원금'], ['중소기업', '청년', '근로자', '미래', '희망', '지원', '사업'], ['청년', '농부', '육성', '지원'], ['산업', '중소기업', '청년', '교통비', '지원', '사업']]
